# API Connections for Routing
Investigation of the following open-access apis:
* https://openrouteservice.org/dev/#/api-docs/v2/
* https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php


## OpenRouteService

* Data Sources
    * OpenStreetMap
    * Crowdsourced updates
* Availability
    * opensource
    * free of charge
* available features
    * multi-modal routing (cars, bikes, walking)
    * time-distance matrix for selecting a favorite among multiple options
* optimizations
    * incorporated methods for tsp, vrp
* interesting oss clients:
    * disaster maps client: disaster management, updated with information
    * classic maps client / new maps client: route planning

In [1]:
from dotenv import dotenv_values
import requests
import folium
import pandas as pd



In [2]:
import openrouteservice as ors

In [3]:
base_path = 'https://api.openrouteservice.org/'
profile_car = 'driving-car'

In [4]:
# environment vars
config = dotenv_values("../.env")
ors_api_key = config['ORS_API_KEY']

### GET Endpoints

In [5]:
# endpoints
directions_endpoint = 'v2/directions/'
geocode_search_endpoint = 'geocode/search'

In [6]:
r1_params = {
    'start':  '8.681495,49.41461',
    'end': '8.687872,49.420318'
}

In [7]:
r1_params_string = ''
for k, v in r1_params.items():
    r1_params_string += f'&{k}={v}'
r1_params_string

'&start=8.681495,49.41461&end=8.687872,49.420318'

In [8]:
r = requests.get(
    f'{base_path}{directions_endpoint}{profile_car}?api_key={ors_api_key}{r1_params_string}'
)


In [9]:
r2_params = {
    'text': 'Namibian Brewery'
}


In [10]:
r2_params_string = ''
for k,v in r2_params.items():
    r2_params_string += f'&{k}={v}'

In [11]:
r2 = requests.get(
    f'{base_path}{geocode_search_endpoint}?api_key={ors_api_key}{r2_params_string}'
)

In [12]:
r2.json()

{'geocoding': {'version': '0.2',
  'attribution': 'https://openrouteservice.org/terms-of-service/#attribution-geocode',
  'query': {'text': 'Namibian Brewery',
   'size': 10,
   'layers': ['venue',
    'street',
    'country',
    'macroregion',
    'region',
    'county',
    'localadmin',
    'locality',
    'borough',
    'neighbourhood',
    'continent',
    'empire',
    'dependency',
    'macrocounty',
    'macrohood',
    'microhood',
    'disputed',
    'postalcode',
    'ocean',
    'marinearea'],
   'private': False,
   'lang': {'name': 'English',
    'iso6391': 'en',
    'iso6393': 'eng',
    'via': 'default',
    'defaulted': True},
   'querySize': 20,
   'parser': 'pelias',
   'parsed_text': {'subject': 'Namibian Brewery',
    'venue': 'Namibian Brewery'}},
  'warnings': ["performance optimization: excluding 'address' layer"],
  'engine': {'name': 'Pelias', 'author': 'Mapzen', 'version': '1.0'},
  'timestamp': 1687179737734},
 'type': 'FeatureCollection',
 'features': [{'t

In [13]:
def get_request_for_ors(
    endpoint: str,
    parameters: dict,
    profile = None,
    json_ize = True
):
    params_string = ''
    for k, v in parameters.items():
        params_string += f'&{k}={v}'
    params_string
    if profile is not None:
        request_url = f'{base_path}{endpoint}{profile}?api_key={ors_api_key}{params_string}'
    else:
        request_url = f'{base_path}{endpoint}?api_key={ors_api_key}{params_string}'
    r = requests.get(request_url)
    if json_ize:
        return r.json()
    return r

In [14]:
r = get_request_for_ors(
    endpoint= directions_endpoint,
    parameters= r1_params,
    profile= profile_car,
    json_ize= True
)

In [22]:
r['features'][0]['geometry']['coordinates']

[[8.681496, 49.414601],
 [8.68147, 49.414599],
 [8.681488, 49.41465],
 [8.681423, 49.415698],
 [8.681423, 49.415746],
 [8.681427, 49.415802],
 [8.681509, 49.416087],
 [8.681656, 49.41659],
 [8.681825, 49.417081],
 [8.681875, 49.417287],
 [8.681881, 49.417392],
 [8.682057, 49.417393],
 [8.682107, 49.41739],
 [8.682461, 49.417389],
 [8.682676, 49.417387],
 [8.683379, 49.41738],
 [8.683595, 49.417372],
 [8.683709, 49.417368],
 [8.685294, 49.417365],
 [8.685359, 49.417365],
 [8.685442, 49.417365],
 [8.685713, 49.41737],
 [8.686407, 49.417365],
 [8.686717, 49.417366],
 [8.686806, 49.417365],
 [8.68703, 49.41736],
 [8.687156, 49.417357],
 [8.687376, 49.417353],
 [8.687466, 49.417351],
 [8.687547, 49.417349],
 [8.688256, 49.417361],
 [8.688802, 49.417381],
 [8.688793, 49.417434],
 [8.68871, 49.418194],
 [8.688647, 49.418464],
 [8.688539, 49.418963],
 [8.688398, 49.41963],
 [8.690104, 49.419828],
 [8.690123, 49.419833],
 [8.689854, 49.420216],
 [8.689652, 49.420514],
 [8.68963, 49.42051],
 [8.

In [15]:
reversed_coordinates = [list(reversed(coord))
                            for coord 
                            in r['features'][0]['geometry']['coordinates']
                        ]
reversed_coordinates

[[49.414601, 8.681496],
 [49.414599, 8.68147],
 [49.41465, 8.681488],
 [49.415698, 8.681423],
 [49.415746, 8.681423],
 [49.415802, 8.681427],
 [49.416087, 8.681509],
 [49.41659, 8.681656],
 [49.417081, 8.681825],
 [49.417287, 8.681875],
 [49.417392, 8.681881],
 [49.417393, 8.682057],
 [49.41739, 8.682107],
 [49.417389, 8.682461],
 [49.417387, 8.682676],
 [49.41738, 8.683379],
 [49.417372, 8.683595],
 [49.417368, 8.683709],
 [49.417365, 8.685294],
 [49.417365, 8.685359],
 [49.417365, 8.685442],
 [49.41737, 8.685713],
 [49.417365, 8.686407],
 [49.417366, 8.686717],
 [49.417365, 8.686806],
 [49.41736, 8.68703],
 [49.417357, 8.687156],
 [49.417353, 8.687376],
 [49.417351, 8.687466],
 [49.417349, 8.687547],
 [49.417361, 8.688256],
 [49.417381, 8.688802],
 [49.417434, 8.688793],
 [49.418194, 8.68871],
 [49.418464, 8.688647],
 [49.418963, 8.688539],
 [49.41963, 8.688398],
 [49.419828, 8.690104],
 [49.419833, 8.690123],
 [49.420216, 8.689854],
 [49.420514, 8.689652],
 [49.42051, 8.68963],
 [49

In [21]:
r['features'][0]['geometry']['coordinates']

[[8.681496, 49.414601],
 [8.68147, 49.414599],
 [8.681488, 49.41465],
 [8.681423, 49.415698],
 [8.681423, 49.415746],
 [8.681427, 49.415802],
 [8.681509, 49.416087],
 [8.681656, 49.41659],
 [8.681825, 49.417081],
 [8.681875, 49.417287],
 [8.681881, 49.417392],
 [8.682057, 49.417393],
 [8.682107, 49.41739],
 [8.682461, 49.417389],
 [8.682676, 49.417387],
 [8.683379, 49.41738],
 [8.683595, 49.417372],
 [8.683709, 49.417368],
 [8.685294, 49.417365],
 [8.685359, 49.417365],
 [8.685442, 49.417365],
 [8.685713, 49.41737],
 [8.686407, 49.417365],
 [8.686717, 49.417366],
 [8.686806, 49.417365],
 [8.68703, 49.41736],
 [8.687156, 49.417357],
 [8.687376, 49.417353],
 [8.687466, 49.417351],
 [8.687547, 49.417349],
 [8.688256, 49.417361],
 [8.688802, 49.417381],
 [8.688793, 49.417434],
 [8.68871, 49.418194],
 [8.688647, 49.418464],
 [8.688539, 49.418963],
 [8.688398, 49.41963],
 [8.690104, 49.419828],
 [8.690123, 49.419833],
 [8.689854, 49.420216],
 [8.689652, 49.420514],
 [8.68963, 49.42051],
 [8.

In [16]:
m = folium.Map(location= (r['features'][0]['bbox'][1], r['features'][0]['bbox'][2]), tiles='cartodbpositron', zoom_start=13)
# m = folium.Map(bounds=r['features'][0]['bbox'])
m

In [20]:
r['features'][0]['bbox']

[8.681423, 49.414599, 8.690123, 49.420514]

In [18]:
folium.PolyLine(
    locations=reversed_coordinates
).add_to(m)

In [19]:
m

### using openrouteservice

In [101]:
client = ors.Client(key=ors_api_key)

In [102]:
coordinates = [[8.681495,49.41461], [8.687872,49.420318]]

In [104]:
route = client.directions(
    coordinates=coordinates,
    profile=profile_car,
    format='geojson',
    # options={"avoid_features": ["steps"]},
    validate=False,
)

In [106]:
m = folium.Map(location= (r['features'][0]['bbox'][1], r['features'][0]['bbox'][2]), tiles='cartodbpositron', zoom_start=13)
reversed_coordinates = [list(reversed(coord))
                            for coord 
                            in r['features'][0]['geometry']['coordinates']
                        ]
folium.PolyLine(
    locations=reversed_coordinates
).add_to(m)
m


In [107]:
route

{'type': 'FeatureCollection',
 'features': [{'bbox': [8.681423, 49.414599, 8.690123, 49.420514],
   'type': 'Feature',
   'properties': {'segments': [{'distance': 1409.2,
      'duration': 282.0,
      'steps': [{'distance': 1.9,
        'duration': 0.5,
        'type': 11,
        'instruction': 'Head west on Gerhart-Hauptmann-Straße',
        'name': 'Gerhart-Hauptmann-Straße',
        'way_points': [0, 1]},
       {'distance': 314.0,
        'duration': 75.3,
        'type': 3,
        'instruction': 'Turn sharp right onto Wielandtstraße',
        'name': 'Wielandtstraße',
        'way_points': [1, 10]},
       {'distance': 500.9,
        'duration': 76.5,
        'type': 1,
        'instruction': 'Turn right onto Mönchhofstraße',
        'name': 'Mönchhofstraße',
        'way_points': [10, 31]},
       {'distance': 251.9,
        'duration': 60.5,
        'type': 0,
        'instruction': 'Turn left onto Erwin-Rohde-Straße',
        'name': 'Erwin-Rohde-Straße',
        'way_points

## Find round trip
_via Optimization_

In [108]:
m = folium.Map(location=[52.521861, 13.40744], tiles='cartodbpositron', zoom_start=13)

# Some coordinates in Berlin
coordinates = [[13.384116, 52.533558], [13.41774, 52.498929], [13.428726, 52.519355], [13.374825, 52.496369], [13.384116, 52.533558]]

# The popup will show the ID in the coordinate list. In a non-optimized waypoint order, the waypoints
# would have been visited from ID 0 to ID 3 sequentially.
for idx, coords in enumerate(coordinates):
    folium.Marker(location=list(reversed(coords)),
                 popup=folium.Popup("ID: {}".format(idx))).add_to(m)

route = client.directions(
    coordinates=coordinates,
    profile='foot-walking',
    format='geojson',
    validate=False,
    optimize_waypoints=True
)

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)
    
m

In [132]:
steps_list = route['features'][0]['properties']['segments'][0]['steps']
steps_list

[{'distance': 129.0,
  'duration': 92.9,
  'type': 11,
  'instruction': 'Head southeast',
  'name': '-',
  'way_points': [0, 4]},
 {'distance': 3.9,
  'duration': 2.8,
  'type': 0,
  'instruction': 'Turn left',
  'name': '-',
  'way_points': [4, 5]},
 {'distance': 30.0,
  'duration': 21.6,
  'type': 1,
  'instruction': 'Turn right',
  'name': '-',
  'way_points': [5, 10]},
 {'distance': 109.4,
  'duration': 78.8,
  'type': 1,
  'instruction': 'Turn right',
  'name': '-',
  'way_points': [10, 13]},
 {'distance': 4.5,
  'duration': 3.2,
  'type': 3,
  'instruction': 'Turn sharp right',
  'name': '-',
  'way_points': [13, 15]},
 {'distance': 12.2,
  'duration': 8.8,
  'type': 2,
  'instruction': 'Turn sharp left',
  'name': '-',
  'way_points': [15, 17]},
 {'distance': 5.1,
  'duration': 3.7,
  'type': 0,
  'instruction': 'Turn left onto Invalidenstraße',
  'name': 'Invalidenstraße',
  'way_points': [17, 18]},
 {'distance': 21.7,
  'duration': 15.6,
  'type': 1,
  'instruction': 'Turn rig

In [127]:
steps_df = pd.DataFrame(steps_list)
steps_df
# steps_list

,distance,duration,type,instruction,name,way_points
0,129.0,92.9,11,Head southeast,-,"[0, 4]"
1,3.9,2.8,0,Turn left,-,"[4, 5]"
2,30.0,21.6,1,Turn right,-,"[5, 10]"
3,109.4,78.8,1,Turn right,-,"[10, 13]"
4,4.5,3.2,3,Turn sharp right,-,"[13, 15]"
5,12.2,8.8,2,Turn sharp left,-,"[15, 17]"
6,5.1,3.7,0,Turn left onto Invalidenstraße,Invalidenstraße,"[17, 18]"
7,21.7,15.6,1,Turn right,-,"[18, 22]"
8,16.6,12.0,0,Turn left,-,"[22, 24]"
9,16.1,11.6,0,Turn left,-,"[24, 26]"


### Find round trips with n Steps

In [159]:
coordinates = [[13.384116, 52.533558], [13.41774, 52.498929], [13.428726, 52.519355], [13.374825, 52.496369], [13.384116, 52.533558]]
index_list = [0,1,2,3,0]

In [205]:
import itertools
permutations = list(itertools.permutations(index_list[1:-1]))
new_permutations = []
for el in permutations:
    el_list = [i for i in el]
    new_permutations.append([0]+ el_list+ [0])
new_permutations
# permutations

[[0, 1, 2, 3, 0],
 [0, 1, 3, 2, 0],
 [0, 2, 1, 3, 0],
 [0, 2, 3, 1, 0],
 [0, 3, 1, 2, 0],
 [0, 3, 2, 1, 0]]

In [221]:
coordinates_permuted = []

for index,el in enumerate(new_permutations):
    new_coords = []
    for i in new_permutations[index]:
        new_coords.append(coordinates[i])
    coordinates_permuted.append(new_coords)

In [226]:
new_permutations[1]

[0, 1, 3, 2, 0]

In [225]:
coordinates_permuted[1]

[[13.384116, 52.533558],
 [13.41774, 52.498929],
 [13.374825, 52.496369],
 [13.428726, 52.519355],
 [13.384116, 52.533558]]

In [244]:
def get_round_trip(coordinates, index_list):
    route_list = []
    route_summary_list = []
    for step_index in range (1, len(index_list)):
        print(f'\tnode {index_list[step_index-1]}: {coordinates[step_index-1]} \t| node {index_list[step_index]}: {coordinates[step_index]}')
        route = client.directions(
            coordinates=[coordinates[step_index-1], coordinates[step_index]],
            profile=profile_car,
            format='geojson',
            validate=False,
        )
        route_list.append(route)
        route_summary_list.append({
            'index': index_list[step_index],
            'distance': route['features'][0]['properties']['segments'][0]['distance'],
            'duration': route['features'][0]['properties']['segments'][0]['duration'],
        })

    return route_list, pd.DataFrame(route_summary_list)

In [230]:
route_list, route_sum_df = get_round_trip(coordinates= coordinates, index_list=index_list)

node 0: [13.384116, 52.533558] 	| node 1: [13.41774, 52.498929]
node 1: [13.41774, 52.498929] 	| node 2: [13.428726, 52.519355]
node 2: [13.428726, 52.519355] 	| node 3: [13.374825, 52.496369]
node 3: [13.374825, 52.496369] 	| node 0: [13.384116, 52.533558]


In [150]:
route_summary_list = []
for idx, el in enumerate(route_list):
    route_summary_list.append({
        'index': idx,
        'distance': el['features'][0]['properties']['segments'][0]['distance'],
        'duration': el['features'][0]['properties']['segments'][0]['duration'],
    })
route_summary_list

[{'index': 0, 'distance': 7884.4, 'duration': 985.3},
 {'index': 1, 'distance': 3037.7, 'duration': 521.1},
 {'index': 2, 'distance': 6234.4, 'duration': 843.3},
 {'index': 3, 'distance': 5196.7, 'duration': 657.8}]

In [152]:
route_summary_df = pd.DataFrame(route_summary_list)
route_summary_df

,index,distance,duration
0,0,7884.4,985.3
1,1,3037.7,521.1
2,2,6234.4,843.3
3,3,5196.7,657.8


In [231]:
print(route_sum_df)

   index  distance  duration
0      1    7884.4     985.3
1      2    3037.7     521.1
2      3    6234.4     843.3
3      0    5196.7     657.8


In [235]:
def get_coords_and_index_list_permutations(
    coordinates: list,
    index_list: list
):
    index_permutations = list(itertools.permutations(index_list[1:-1]))
    new_permutations = []
    for el in index_permutations:
        el_list = [i for i in el]
        new_permutations.append([0]+ el_list+ [0])
    
    coordinates_permuted = []

    for idx, el in enumerate(new_permutations):
        new_coords = []
        for i in new_permutations[idx]:
            new_coords.append(coordinates[i])
        coordinates_permuted.append(new_coords)
    return coordinates_permuted, new_permutations

In [239]:
coordinates_permuted, index_permuted = get_coords_and_index_list_permutations(
    coordinates= coordinates,
    index_list= [0,1,2,3,0]
)

In [245]:
permutations_route_list = []

for idx, el in enumerate(index_permuted):
    print(f'index: {idx}, index_permuted: {index_permuted[idx]}, coords_permuted: {coordinates_permuted[idx]} ')
    route_list, route_sum_df = get_round_trip(coordinates= coordinates_permuted[idx], index_list=index_permuted[idx])
    permutations_route_list.append(
        {
            'index': idx,
            'route_list': route_list,
            'route_sum_df': route_sum_df
        }
    )

index: 0, index_permuted: [0, 1, 2, 3, 0], coords_permuted: [[13.384116, 52.533558], [13.41774, 52.498929], [13.428726, 52.519355], [13.374825, 52.496369], [13.384116, 52.533558]] 
	node 0: [13.384116, 52.533558] 	| node 1: [13.41774, 52.498929]
	node 1: [13.41774, 52.498929] 	| node 2: [13.428726, 52.519355]
	node 2: [13.428726, 52.519355] 	| node 3: [13.374825, 52.496369]
	node 3: [13.374825, 52.496369] 	| node 0: [13.384116, 52.533558]
index: 1, index_permuted: [0, 1, 3, 2, 0], coords_permuted: [[13.384116, 52.533558], [13.41774, 52.498929], [13.374825, 52.496369], [13.428726, 52.519355], [13.384116, 52.533558]] 
	node 0: [13.384116, 52.533558] 	| node 1: [13.41774, 52.498929]
	node 1: [13.41774, 52.498929] 	| node 3: [13.374825, 52.496369]
	node 3: [13.374825, 52.496369] 	| node 2: [13.428726, 52.519355]
	node 2: [13.428726, 52.519355] 	| node 0: [13.384116, 52.533558]
index: 2, index_permuted: [0, 2, 1, 3, 0], coords_permuted: [[13.384116, 52.533558], [13.428726, 52.519355], [13.4

In [277]:
permutations_route_list[0]['route_sum_df']

,index,distance,duration
0,1,7884.4,985.3
1,2,3037.7,521.1
2,3,6234.4,843.3
3,0,5196.7,657.8


### Matrix
* distances of every node to start/end point

In [278]:
m = folium.Map(location=[52.521861, 13.40744], tiles='cartodbpositron', zoom_start=13)

# Some coordinates in Berlin
coordinates = [[13.384116, 52.533558], [13.428726, 52.519355], [13.41774, 52.498929], [13.374825, 52.496369]]

matrix = client.distance_matrix(
    locations=coordinates,
    profile=profile_car,
    metrics=['distance', 'duration'],
    validate=False,
)

for marker in coordinates:
    folium.Marker(location=list(reversed(marker))).add_to(m)

print("Durations in secs: {}\n".format(matrix['durations']))
print("Distances in m: {}".format(matrix['distances']))

m

Durations in secs: [[0.0, 663.22, 985.28, 691.28], [647.47, 0.0, 514.75, 843.28], [905.03, 521.11, 0.0, 503.28], [657.76, 833.93, 513.01, 0.0]]

Distances in m: [[0.0, 4044.07, 7884.37, 5455.42], [4071.65, 0.0, 2716.73, 6234.41], [7755.53, 3037.68, 0.0, 4007.69], [5196.67, 5953.24, 3753.86, 0.0]]


In [279]:
matrix

{'durations': [[0.0, 663.22, 985.28, 691.28],
  [647.47, 0.0, 514.75, 843.28],
  [905.03, 521.11, 0.0, 503.28],
  [657.76, 833.93, 513.01, 0.0]],
 'distances': [[0.0, 4044.07, 7884.37, 5455.42],
  [4071.65, 0.0, 2716.73, 6234.41],
  [7755.53, 3037.68, 0.0, 4007.69],
  [5196.67, 5953.24, 3753.86, 0.0]],
 'destinations': [{'location': [13.38395, 52.533489],
   'snapped_distance': 13.61},
  {'location': [13.428816, 52.519341], 'snapped_distance': 6.3},
  {'location': [13.417828, 52.498956], 'snapped_distance': 6.69},
  {'location': [13.37311, 52.497606], 'snapped_distance': 179.97}],
 'sources': [{'location': [13.38395, 52.533489], 'snapped_distance': 13.61},
  {'location': [13.428816, 52.519341], 'snapped_distance': 6.3},
  {'location': [13.417828, 52.498956], 'snapped_distance': 6.69},
  {'location': [13.37311, 52.497606], 'snapped_distance': 179.97}],
 'metadata': {'attribution': 'openrouteservice.org | OpenStreetMap contributors',
  'service': 'matrix',
  'timestamp': 1686586414931,
 